In [163]:
import pandas as pd
import numpy as np
import os
from scipy.sparse import csr_matrix

In [178]:
files = os.listdir("fullyAnnotatedSequences/")
df = pd.read_csv('sequences_features/'+ files[0][0:-5] + '.dat')
i = 1
df['sequence'] = i
lengths = [len(df)]
for file in files[1:]:
    i += 1
    small_df = pd.read_csv('sequences_features/'+ file[0:-5] + '.dat')
    lengths.append(len(small_df))
    small_df['sequence'] = i
    df = df.append(small_df)

In [199]:
df['unique_ID'] = df["sequence"].map(str) + df["1:sentId"]  
frame_features = df[['29:posFramesFired',
 '30:negFramesFired',
 '31:otherFramesFired',
 '32:posFramewrtCharFired',
 '33:negFramewrtCharFired',
 '34:otherFrameswrtCharFired', 'unique_ID']]
df

,1:sentId,2:AreTeam,3:CountPosSpanWords,4:CountNegSpanWords,5:MeanPol_PosActsToEachOther_Choi,6:MeanPol_NegActsToEachOther_Choi,7:Is_PosActsToEachOther_LiuPol,8:Is_NegActsToEachOther_LiuPol,9:Is_PosActsToEachOther_MPQAPol,10:Is_NegActsToEachOther_MPQAPol,...,27:Is_PosAdverbs_Acts1Char_MPQAPol,28:Is_NegAdverbs_Acts1Char_MPQAPol,29:posFramesFired,30:negFramesFired,31:otherFramesFired,32:posFramewrtCharFired,33:negFramewrtCharFired,34:otherFrameswrtCharFired,sequence,unique_ID
0,s0,0,1,1,0.000000,0.000000,0,0,0,0,...,0,0,2,2,1,1,2,1,1,1s0
1,s9,0,0,2,0.000000,0.000000,0,0,0,0,...,0,0,1,1,0,0,0,0,1,1s9
2,s10,0,0,0,0.000000,0.000000,0,0,0,0,...,0,0,2,1,0,0,0,0,1,1s10
3,s12,0,0,1,0.000000,0.000000,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1s12
4,s13,0,0,3,0.000000,0.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1s13
5,s14,0,0,0,0.000000,0.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1s14
6,s17,0,0,0,0.000000,0.000000,0,0,0,0,...,0,0,1,0,0,1,0,0,1,1s17
7,s21,0,0,0,0.000000,0.000000,0,0,0,0,...,0,0,0,2,1,0,2,1,1,1s21
8,s23,0,0,0,0.000000,0.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1s23
9,s24,0,0,0,0.000000,0.000000,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1s24


In [3]:
features_df = df[df.columns[1:]]
features_matrix = np.asmatrix(features_df)
features_sparse = csr_matrix(features_matrix)

In [91]:
files = os.listdir("fullyAnnotatedSequences/")
df = pd.read_csv('fullyAnnotatedSequences/'+ files[0], sep = ':::')
df['pairID'] = 0
lengths = [len(df)]
i = 1
for file in files[1:]:
    small_df = pd.read_csv('fullyAnnotatedSequences/'+ file, sep = ':::')
    small_df['pairID'] = i
    i+= 1
    lengths.append(len(small_df))
    df = df.append(small_df)

/Users/sushmitavgopalan/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app
/Users/sushmitavgopalan/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [128]:
def split_stuff(df):
    y = np.asarray(df['manualLabel'])
    lengths = np.asarray(df.groupby('pairID').count()['manualLabel'].tolist())
    features_df = df[df.columns[0:-4]]
    features_matrix = np.asmatrix(features_df)
    features_sparse = csr_matrix(features_matrix)
    X = features_sparse
    return X,y,lengths

In [93]:
full = pd.concat([features_df,df], axis = 1)
from sklearn.model_selection import train_test_split
train, test = train_test_split(full, test_size = 0.2)

In [94]:
X_train, y_train, lengths_train = split_stuff(train)
X_test, y_test, lengths_test = split_stuff(test)

In [166]:
frames = []
for colname in full.columns:
    if 'Frame' in colname:
        frames.append(colname)
frames

['29:posFramesFired',
 '30:negFramesFired',
 '31:otherFramesFired',
 '32:posFramewrtCharFired',
 '33:negFramewrtCharFired',
 '34:otherFrameswrtCharFired']

In [203]:
from __future__ import print_function
import fileinput
from glob import glob
import sys

from seqlearn.datasets import load_conll
from seqlearn.evaluation import bio_f_score
from seqlearn.perceptron import StructuredPerceptron
from sklearn.metrics import accuracy_score
from seqlearn.hmm import MultinomialHMM
from seqlearn.base import BaseSequenceClassifier

In [208]:
#np.random.seed(324)
clf = MultinomialHMM()
print("Training %s" % clf)
clf.fit(X_train, y_train, lengths_train)

y_pred = clf.predict(X_test, lengths_test)
print("Accuracy: %.3f" % (100 * accuracy_score(y_test, y_pred)))



Training MultinomialHMM(alpha=0.01, decode='viterbi')
Accuracy: 57.143


In [144]:
ours = pd.read_csv('fully_features.csv')

In [200]:
ours_full

,Unnamed: 0,sequence,sid,pos_acts_toeachother,neg_acts_toeachother,pos_acts_together,neg_acts_together,pos_char1_acts,neg_char1_acts,pos_char2_acts,neg_char2_acts,pos,neg,unique_ID,manualLabel
0,0,1,s0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1s0,n
1,3,1,s12,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1s12,n
2,5,1,s14,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1s14,n
3,9,1,s24,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1s24,n
4,10,2,s0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2s0,n
5,12,2,s14,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2s14,n
6,16,3,s0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,3s0,n
7,18,3,s14,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3s14,n
8,19,3,s15,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3s15,n
9,24,4,s15,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4s15,n


In [193]:
ours['unique_ID'] = ours["sequence"].map(str) + ours["sid"]

In [194]:
full['unique_ID'] = full["pairID"].map(str) + full["sid"]

In [195]:
full_subset = full[['unique_ID','manualLabel']]


In [198]:
len(ours)
len(frame_features)
len(pd.merge(ours,frame_features))

0

In [184]:
ours_full = pd.merge(ours, full_subset)
#ours_full = pd.merge(ours_full,features_df)

feature_vars = ['pos_acts_toeachother', 'neg_acts_toeachother',
               'pos_acts_together', 'neg_acts_together',
               'pos_char1_acts','neg_char1_acts', 
               'pos_char2_acts','neg_char2_acts',
               'pos','neg'].extend(frames)


In [150]:
def split_stuff(df):
    y = np.asarray(df['manualLabel'])
    lengths = np.asarray(df.groupby('sequence').count()['manualLabel'].tolist())
    features_df = df[feature_vars]
    features_matrix = np.asmatrix(features_df)
    features_sparse = csr_matrix(features_matrix)
    X = features_sparse
    return X,y,lengths

In [151]:
train, test = train_test_split(ours_full, test_size = 0.2)
X_train, y_train, lengths_train = split_stuff(train)
X_test, y_test, lengths_test = split_stuff(test)

In [201]:
#clf = MultinomialHMM()
from seqlearn
clf = BaseSequenceClassifier()
print("Training %s" % clf)
clf.fit(X_train, y_train, lengths_train)

y_pred = clf.predict(X_test, lengths_test)
print("Accuracy: %.3f" % (100 * accuracy_score(y_test, y_pred)))

NameError: name 'BaseSequenceClassifier' is not defined